In [2]:
import transformers 
import datasets
import torch
import logging

c:\Users\dama_\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cpu


In [4]:
# Preprocessing
# sep_token = '<sep>'
dataset_name = "squad_v2"
model_type="bert"
model_name= "bert-base-cased"
models_dir = "saved_models/bert_base_cased-squad2_mod"
# checkpoint = 'roberta-base'
# max_input_length = 512
# max_target_length = 64

# ## Training
# learning_rate = 3e-5
# num_epochs = 7

In [5]:
dataset = datasets.load_dataset(dataset_name)

Found cached dataset squad_v2 (C:/Users/dama_/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)
100%|██████████| 2/2 [00:00<00:00, 166.63it/s]


In [6]:
train_dataset=dataset['train']
valid_dataset=dataset['validation']
print(train_dataset)
print(valid_dataset)

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 130319
})
Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 11873
})


In [7]:
torch.save(train_dataset, 'train_data.pt')
torch.save(valid_dataset, 'valid_data.pt')

In [8]:
# transformers.BertForQuestionAnswering
model =transformers.BertForQuestionAnswering.from_pretrained(model_name)
tokenizer = transformers.BertTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and a

In [9]:
# Configure the model 
model_args = transformers.TrainingArguments(
    output_dir=models_dir,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size=16
    )
# model_args.train_batch_size = 128
# model_args.eval_batch_size=64
# model_args.evaluate_during_training = True
# model_args.n_best_size=3
model_args.num_train_epochs=5
model_args.overwrite_output_dir:True
model_args.evaluation_strategy="steps"
model_args.logging_steps=100
model_args.save_steps=500
model_args.learning_rate=3e-5

In [10]:
logger = logging.getLogger(__name__)

# Initialize our Trainer
trainer = transformers.Trainer(
    model=model,
    args=model_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer =tokenizer
    # data_collator=T2TDataCollator()
)

In [11]:
if model.device.type == 'cuda':
    print('Model is on GPU')
else:
    print('Model is on CPU')

Model is on CPU


In [17]:
# Tokenize the examples
def tokenize_examples(examples):
    features = []
    for i in len(examples):
        context = examples[i]['context']
        question = examples[i]['question']
        inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors='pt')
        # answer_start = torch.tensor(examples[i]start_position)
        # answer_end = torch.tensor(example.end_position)
        features.append({'input_ids': inputs['input_ids'][0], 'attention_mask': inputs['attention_mask'][0], 
                         'token_type_ids': inputs['token_type_ids'][0], 
                        #  'start_positions': answer_start, 
                        #  'end_positions': answer_end
                         })
    return features

In [ ]:
train_features = tokenize_examples(train_dataset)
val_features = tokenize_examples(valid_dataset)

In [12]:
trainer.train()

c:\Users\dama_\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:
Traceback (most recent call last):
  File "c:\Users\dama_\AppData\Local\Programs\Python\Python311\Lib\site-packages\wandb\sdk\wandb_init.py", line 1141, in init
    wi.

Error: An unexpected error occurred

In [ ]:
torch.cuda.is_available()